In [ ]:
!pip install pyannote.audio

In [ ]:
!pip install pydub

In [ ]:
from pyannote.audio import Pipeline
import torch
import time
start = time.time()
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=""
)
# Move the pipeline to GPU if available
if torch.cuda.is_available():
    pipeline = pipeline.to(torch.device("cuda"))

    # apply pretrained pipeline
    diarization = pipeline("")

    # print the result
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
    end = time.time()
    print(start-end)
else:
    print("Cannot do!")

In [ ]:
from pyannote.audio import Pipeline
import torch
import time
import os
from pydub import AudioSegment

# Function to split the audio into smaller chunks
def split_audio(input_audio, chunk_duration):
    chunk_length = chunk_duration * 1000  # Convert chunk duration from seconds to milliseconds
    chunks = []
    start = 0
    end = chunk_length
    while end <= len(input_audio):
        chunks.append(input_audio[start:end])
        start = end
        end += chunk_length
    # Add the last chunk (which may be shorter than chunk_length)
    if start < len(input_audio):
        chunks.append(input_audio[start:])
    return chunks

start_time = time.time()

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=""
)

# Move the pipeline to GPU if available
if torch.cuda.is_available():
    pipeline = pipeline.to(torch.device("cuda"))

# Load the hour-long audio file
input_audio = AudioSegment.from_wav("")

# Define the output directory for audio chunks
output_dir = "/kaggle/working/audio_chunks/"

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Define the duration of each chunk in seconds
chunk_duration = 300  # 5 minutes

# Split the audio into smaller chunks
audio_chunks = split_audio(input_audio, chunk_duration)

# Process each chunk
for i, chunk in enumerate(audio_chunks):
    chunk_output_path = os.path.join(output_dir, f"chunk_{i+1}.wav")
    chunk.export(chunk_output_path, format="wav")  # Export the chunk to a WAV file
    print(f"Processing chunk {i+1}/{len(audio_chunks)}...")
    
    # Apply pretrained pipeline
    diarization = pipeline(chunk_output_path)

    # Print the result
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

# Calculate and print the execution time
end_time = time.time()
print("Total execution time:", end_time - start_time, "seconds")


In [ ]:
from pyannote.audio import Pipeline
import torch
import time
import os
from pydub import AudioSegment

# Function to split audio into smaller chunks
def split_audio(input_audio, chunk_duration):
    chunk_length = chunk_duration * 1000  # Convert chunk duration from seconds to milliseconds
    chunks = []
    start = 0
    end = chunk_length
    while end <= len(input_audio):
        chunks.append(input_audio[start:end])
        start = end
        end += chunk_length
    if start < len(input_audio):
        chunks.append(input_audio[start:])
    return chunks

# Function to perform diarization on audio chunks
def diarize_audio_chunks(audio_chunks):
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization-3.1",
        use_auth_token=""
    )

    if torch.cuda.is_available():
        pipeline = pipeline.to(torch.device("cuda"))

    for i, chunk in enumerate(audio_chunks):
        chunk_file_path = f"chunk_{i+1}.wav"
        chunk.export(chunk_file_path, format="wav")
        print(f"Processing chunk {i + 1}/{len(audio_chunks)}...")
        diarization = pipeline(chunk_file_path)
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
        os.remove(chunk_file_path)  # Remove temporary file after processing

# Read the WAV file
audio_file_path = ""
audio_file = AudioSegment.from_wav(audio_file_path)

# Define chunk duration in seconds
chunk_duration = 60  # 1 minute

# Split the audio into smaller chunks
audio_chunks = split_audio(audio_file, chunk_duration)

# Perform diarization on audio chunks
diarize_audio_chunks(audio_chunks)
